<a href="https://colab.research.google.com/github/wkdtjrrb/SPIDER/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd '/content/drive/MyDrive/TeamSpider/Titanic'

/content/drive/MyDrive/TeamSpider/Titanic


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LinearRegression
from collections import Counter
from mpl_toolkits import mplot3d
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

In [ ]:
path = './train.csv'
df = pd.read_csv(path)

In [ ]:
path = './test.csv'
test_df = pd.read_csv(path)

In [ ]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [ ]:
ob_data = df.select_dtypes(include = 'object')
num_data = df.select_dtypes(exclude = 'object')

**데이터 전처리 및 상관관계 분석**

1. 데이터 범주화

1. 이상치 처리 ( 데이터를 살펴본 결과 이상치라고 할 만한 것이 없다고 판단 )

2. 결측값 처리 ( 칼럼에 따라 다른 방식 적용 )

3. 데이터 변환 ( categorical to numerical )

4. 상관관계 분석 ( 데이터 변환 전 categorical data의 상관관계 분석 필요 )

5. 데이터 스케일링

In [ ]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Age(나이) : 10대 별로 범주화

Cabin(객실) : 결측값을 특정한 값으로 채울 수 없으므로 결측값이 아닌 데이터에 대해서만 분석을 진행

Embarked(승선항구)

Sibsp = 형제자매 수 / Parch = 부모자식 수

Pclass = 승객 등급

In [ ]:
#'Age'칼럼 처리
def age_converter(age):
    if age <= 9:
        return 1
    elif age <= 19:
        return 10
    elif age <= 29:
        return 20
    elif age <= 39:
        return 30
    elif age <= 49:
        return 40
    elif age <= 59:
        return 50
    elif age <= 69:
        return 60
    elif age <= 79:
        return 70
    elif age <= 89:
        return 80
    elif age <=99:
        return 90
    else:
        return -1

# apply 메서드를 사용하여 age column에 함수 적용
df['Age'] = df['Age'].apply(age_converter)
test_df['Age'] = test_df['Age'].apply(age_converter)

Cabin 처리
1. 채워지지 않은 것은 Unknown으로 대체
2. 앞의 문자로 처리

In [ ]:
df["Cabin"].fillna('Unknown',inplace = True)
test_df["Cabin"].fillna('Unknown',inplace = True)

In [ ]:
#Cabin의 경우 앞 알파벳으로 구분
df["Cabin"] = df["Cabin"].str[0]
test_df["Cabin"] = test_df["Cabin"].str[0]

In [ ]:
df['Cabin'].value_counts()

U    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: Cabin, dtype: int64

Ticket 처리

In [ ]:
# pattern : 알파벳만 뽑아내기 위함
pattern = r'([a-zA-Z]+)'

# 패턴을 사용하여 문자열 추출 및 숫자로만 이루어진 cabin은 NUM으로 통일
df['Ticket'] = df['Ticket'].str.extract(pattern, expand=False)
df['Ticket'].fillna('NUM',inplace = True)
test_df['Ticket'] = test_df['Ticket'].str.extract(pattern, expand=False)
test_df['Ticket'].fillna('NUM',inplace = True)

In [ ]:
'''
ticket_counts = df["Ticket"].value_counts()
top_ticket = list(ticket_counts[1:9].index)

# 상위 1~9개 범주 이외는 "Other"로 합침
df["Ticket"] = df["Ticket"].apply(lambda x: x if x in top_ticket else "Other")
sns.histplot(data=df, x="Ticket", hue="Survived", multiple="fill", discrete=True,stat = 'probability', fill = True)
'''

'\nticket_counts = df["Ticket"].value_counts()\ntop_ticket = list(ticket_counts[1:9].index)\n\n# 상위 1~9개 범주 이외는 "Other"로 합침\ndf["Ticket"] = df["Ticket"].apply(lambda x: x if x in top_ticket else "Other")\nsns.histplot(data=df, x="Ticket", hue="Survived", multiple="fill", discrete=True,stat = \'probability\', fill = True)\n'

Name 처리

In [ ]:
df["Name"] = df["Name"].str.extract('([A-Za-z]+)\.')
test_df["Name"] = test_df["Name"].str.extract('([A-Za-z]+)\.')

In [ ]:
df['Name'].value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: Name, dtype: int64

In [ ]:
'''
name_counts = df["Name"].value_counts()
top_names = list(name_counts[:4].index)

# 상위 4개 범주 이외는 "Other"로 합침
df["Name"] = df["Name"].apply(lambda x: x if x in top_names else "Other")

#이름별 생존 비율의 확인
sns.histplot(data=df, x="Name", hue="Survived", multiple="fill", discrete=True,stat = 'probability', fill = True)
'''

'\nname_counts = df["Name"].value_counts()\ntop_names = list(name_counts[:4].index)\n\n# 상위 4개 범주 이외는 "Other"로 합침\ndf["Name"] = df["Name"].apply(lambda x: x if x in top_names else "Other")\n\n#이름별 생존 비율의 확인\nsns.histplot(data=df, x="Name", hue="Survived", multiple="fill", discrete=True,stat = \'probability\', fill = True)\n'

In [ ]:
'''
sns.catplot(x='Survived', col='Embarked', kind='count', data=df)
'''

"\nsns.catplot(x='Survived', col='Embarked', kind='count', data=df)\n"

**결측값 처리**

결측값이 없는 칼럼들을 이용해 KNN기법으로 결측값 대체

1. Embarked 결측값 처리

-KNN에서 사용하는 칼럼 Pclass, Name, Sex, Age, Ticket, Cabin

-첫번째 결측값 : 가장 빈도수가 높은 S를 선택

-2번째 결측값 : Fare가 가까운 쪽인 C를 선택

2. Age 결측값 처리

 - Age는 이름과 성별로만 선택

In [ ]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64

In [ ]:
test_df.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin          0
Embarked       0
dtype: int64

In [ ]:
missing_embarked = df[df['Embarked'].isnull()]

# 결측값이 있는 모든 행에 대해 같은 다른 행 찾기
matching_rows_list = []
for index, row in missing_embarked.iterrows():
    matching_rows = df[(df['PassengerId'] != row['PassengerId']) &
                       (df['Ticket']==row['Ticket']) &
                       (df['Name'] == row['Name']) &
                       (df['Sex'] == row['Sex']) &
                       (df['Age'] == row['Age']) &
                       (df['Survived'] == row['Survived'])]
    matching_rows_list.append(matching_rows)

matching_rows_df = pd.concat(matching_rows_list)

print(matching_rows_df)

     PassengerId  Survived  Pclass  Name     Sex  Age  SibSp  Parch Ticket  \
79            80         1       3  Miss  female   30      0      0    NUM   
123          124         1       2  Miss  female   30      0      0    NUM   
215          216         1       1  Miss  female   30      1      0    NUM   
218          219         1       1  Miss  female   30      0      0    NUM   
257          258         1       1  Miss  female   30      0      0    NUM   
318          319         1       1  Miss  female   30      0      2    NUM   
322          323         1       2  Miss  female   30      0      0    NUM   
387          388         1       2  Miss  female   30      0      0    NUM   
412          413         1       1  Miss  female   30      1      0    NUM   
520          521         1       1  Miss  female   30      0      0    NUM   
576          577         1       2  Miss  female   30      0      0    NUM   
747          748         1       2  Miss  female   30      0    

In [ ]:
null_idx_1 = df[df['Embarked'].isnull()].index[0]
df.loc[null_idx_1, 'Embarked'] = 'S'
null_idx_2 = df[df['Embarked'].isnull()].index[0]
df.loc[null_idx_2,'Embarked'] = 'C'

In [ ]:
for i in range(len(df)):
    if df.loc[i, 'Age'] == -1:
        same_condition = ((df['Name'] == df.loc[i, 'Name']) &
                         (df['Sex'] == df.loc[i, 'Sex']))
        mode_age = df[same_condition]['Age'].mode()
        if len(mode_age) > 0:
            df.loc[i, 'Age'] = mode_age[0]

In [ ]:
for i in range(len(test_df)):
    if test_df.loc[i, 'Age'] == -1:
        same_condition = ((df['Name'] == test_df.loc[i, 'Name']) &
                         (df['Sex'] == test_df.loc[i, 'Sex']))
        mode_age = df[same_condition]['Age'].mode()
        if len(mode_age) > 0:
            test_df.loc[i, 'Age'] = mode_age[0]

In [ ]:
df['Age'].value_counts()

20    375
30    184
10    102
40     89
1      66
50     49
60     19
70      6
80      1
Name: Age, dtype: int64

In [ ]:
test_df['Age'].value_counts()

20    196
30     75
40     46
10     41
1      24
50     22
60     13
70      1
Name: Age, dtype: int64

**catergorical to numerical**


In [ ]:
df.drop(['Ticket','Cabin','Name','PassengerId'],axis = 1,inplace = True)
test_df.drop(['Ticket','Cabin','Name'],axis = 1,inplace = True)

In [ ]:
df['Embarked'].value_counts()

S    645
C    169
Q     77
Name: Embarked, dtype: int64

In [ ]:
df['Sex'] = df['Sex'].map({'male' : 0, 'female' : 1})
df['Embarked'] = df['Embarked'].map({'S' : 0 , 'C' : 1, 'Q' : 3})
test_df['Sex'] = test_df['Sex'].map({'male' : 0, 'female' : 1})
test_df['Embarked'] = test_df['Embarked'].map({'S' : 0 , 'C' : 1, 'Q' : 3})

In [ ]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,20,1,0,7.2500,0
1,1,1,1,30,1,0,71.2833,1
2,1,3,1,20,0,0,7.9250,0
3,1,1,1,30,1,0,53.1000,0
4,0,3,0,30,0,0,8.0500,0


In [ ]:
df.dtypes

Survived      int64
Pclass        int64
Sex           int64
Age           int64
SibSp         int64
Parch         int64
Fare        float64
Embarked      int64
dtype: object

In [ ]:
mean = test_df['Fare'].mode()[0]
test_df['Fare'].fillna(mean,inplace = True)

In [ ]:
# train data 준비
X = df.drop('Survived',axis = 1)
y = df['Survived']

In [ ]:
# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#데이터 스케일링
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#로지스틱 회귀
clf = LogisticRegression(penalty = 'l2',solver = 'liblinear')
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print('Logistic Regression Accuracy:', score)

Logistic Regression Accuracy: 0.7985074626865671


In [ ]:
#KNN
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X,y)
score = knn.score(X_test, y_test)
print('KNN Accuracy:', score)

KNN Accuracy: 0.7089552238805971


In [ ]:
#SVM
svm_model = svm.SVC(kernel='rbf')
svm_model.fit(X_train, y_train)
score = svm_model.score(X_test, y_test)
print('SVM Accuracy:', score)

SVM Accuracy: 0.8171641791044776


In [ ]:
#Random Forest
rf_model = RandomForestRegressor(n_estimators=700,
                               max_depth=9,
                               min_samples_split=2,
                               min_samples_leaf=1,max_features = 'auto')
rf_model.fit(X_train,y_train)
score = rf_model.score(X_test, y_test)
print('Random Forest Accuracy:',score)

Random Forest Accuracy: 0.48700582340094967


In [ ]:
Id_col = test_df['PassengerId']

In [ ]:
test_df.drop(['PassengerId'],axis = 1,inplace = True)

In [ ]:
test_pred = svm_model.predict(test_df)

In [ ]:
test_pred_df = pd.DataFrame(test_pred,columns = ['Survived'])
sub_df = pd.concat([Id_col,test_pred_df],axis = 1)

In [ ]:
sub_df.to_csv('sub_df3.csv',index = False)

**1번째 제출 점수: 0.75598**

**2번째 제출 점수: 0.76076**

**수정해야할 것**

1. cross validation

2. final train with full data

3. filling missing value with recognizing the relation between age and others